### 1

![4-1](./img/4-1.jpg)

### 2

![4-2](./img/4-2.jpg)

### 3

![4-3](./img/4-3.jpg)

In [1]:
import numpy as np
x=np.array([1.0,2.0,3.0])
w=np.array([1.0,2.0,3.0])
y_hat=x*w
y=np.array([2.0,4.0,6.0])
s=y_hat-y
loss=np.square(s)
rloss_rw=np.diff(loss)/np.diff(s)*np.diff(s)/np.diff(y_hat)*\
            np.diff(y_hat)/np.diff(w)
print("Computed backprop: ",rloss_rw)

Computed backprop:  [-1.  9.]


### 4

![4-4](./img/4-4.jpg)

### 5

In [2]:
import torch
import numpy as np

w1=2
w2=2
b=1
x=np.array([1.0,2.0,3.0])
x_2=x**2
x_data=np.vstack([x,x_2])
y_data=x_2*w2+x*w1+b

In [3]:
w1=torch.tensor([1.0],requires_grad=True)
w2=torch.tensor([1.0],requires_grad=True)
# our model forward pass
def forward(x):
    return x[0]*w1+x[1]*w2+b

#loss function
def loss(y_hat,y):
    return (y_hat-y)**2

#before training
print("predict (before training)", 4, forward([4,4**2]).item())

for epoch in range(10):
    for x_val, y_val in zip(x_data.T,y_data):
        y_pred = forward(x_val)
        l = loss(y_pred, y_val)
        l.backward() #Back propagate to update weights
        w1.data=w1.data-0.01*w1.grad.item()
        w2.data=w1.data-0.01*w2.grad.item()
        
        print("\tgrad : ",x_val, y_val, w1.grad.item(), w2.grad.item())
        
        # manually set to zero
        w1.grad.data.zero_()
        w2.grad.data.zero_()
    print(f"Epoch: {epoch} | Loss: {l.item()}")
# after training
print("prediction (after training)", 4, forward([4,4**2]).item())

predict (before training) 4 21.0
	grad :  [1. 1.] 5.0 -4.0 -4.0
	grad :  [2. 4.] 13.0 -22.400001525878906 -44.80000305175781
	grad :  [3. 9.] 25.0 -28.79999542236328 -86.39998626708984
Epoch: 0 | Loss: 23.039993286132812
	grad :  [1. 1.] 5.0 -0.06400012969970703 -0.06400012969970703
	grad :  [2. 4.] 13.0 -10.726402282714844 -21.452804565429688
	grad :  [3. 9.] 25.0 -12.902400970458984 -38.70720291137695
Epoch: 1 | Loss: 4.624220848083496
	grad :  [1. 1.] 5.0 -0.0701436996459961 -0.0701436996459961
	grad :  [2. 4.] 13.0 -5.037670135498047 -10.075340270996094
	grad :  [3. 9.] 25.0 -6.0788726806640625 -18.236618041992188
Epoch: 2 | Loss: 1.0264637470245361
	grad :  [1. 1.] 5.0 -0.032088279724121094 -0.032088279724121094
	grad :  [2. 4.] 13.0 -2.3680877685546875 -4.736175537109375
	grad :  [3. 9.] 25.0 -2.8571090698242188 -8.571327209472656
Epoch: 3 | Loss: 0.22675201296806335
	grad :  [1. 1.] 5.0 -0.015102386474609375 -0.015102386474609375
	grad :  [2. 4.] 13.0 -1.113128662109375 -2.22625

### Summarize the properties and behaviors of the optimizers

#### SGD
full-batch가 아닌 mini batch로 학습을 진행하는 방식

#### Adagrad
학습을 통해 큰 변동이 있는 가중치에 대해서는 학습률을 감소시키고, 아직 가중치의 변동이 별로 없었던 가중치는 학습률을 증가시켜서 학습을 진행하는 방식

#### RMSProp
Adagrad는 간단한 convex function에서는 잘 동작하지만, 복잡한 다차원 곡면 function에서는 global minimum에 도달하기 전에 학습률이 0으로 수렴할 수 있다. 따라서 RMSProp에서 이를 보완하였다.

a) 가중치의 기울기를 단순 누적시키는 것이 아니라, 최신 기울기들이 더 반영되도록 한다.

b) hyper parameter p를 추가하여 h가 무한히 커지지 않게 한다.(h는 가중치 기울기 제곱들을 더해가는 term이다.)

#### Adam
Momentum과 RMSProp을 융합한 방법이다.

#### Adamax
Adam의 LR을 조절하는 부분의 L2 norm을 Lp norm으로 교체한 방식이다.

#### LBFGS
LBFGS는 Limited-memory quasi-Newton methods의 한 예시로써, Hessian 행렬을 계산하거나 저장하기 위한 비용이 합리적이지 않을 경우 유용하게 사용된다. 이 방법은 밀도가 높은 n×n의 Hessian 행렬을 저장하는 대신 n차원의 벡터 몇 개만을 유지하여 Hessian 행렬을 추정(approximation)하는 방식이다.

#### Rprop
Gradient의 부호만을 사용하여 Gradient를 업데이트 하는 방식이다.

#### ASGD
n 스텝의 가중치를 평균하여 가중치를 업데이트하는 방식이다.

### Summarize pytorch tutorial

Pytorch는 autograd 패키지를 통해 자동미분 연산을 지원해줘 역전파 단계의 연산을 자동화 할 수 있다.

또한 사용자 정의 모델을 nn.Module을 상속 받아 간단하게 신경망 구성히 가능하다.